In [1]:
%pip install easyocr sentence-transformers transformers torch torchvision opencv-python pillow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import cv2
import numpy as np
import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import easyocr
import warnings

warnings.filterwarnings('ignore')


class ChildcareDocumentClassifier:
    def __init__(self, similarity_threshold: float = 0.70):
        print("📥 지정 사전학습 모델 로딩 중...")
        print("  - MPNet 문장 임베딩 로딩...")

        self.mpnet_tokenizer = AutoTokenizer.from_pretrained(
            "sentence-transformers/all-mpnet-base-v2"
        )
        self.mpnet_model = AutoModel.from_pretrained(
            "sentence-transformers/all-mpnet-base-v2"
        )
        self.mpnet_model.eval()

        self.threshold = similarity_threshold
        print("✅ 모든 모델 로딩 완료!")

        # OCR 리더 (EasyOCR, 한·영)
        self.ocr_reader = easyocr.Reader(["ko", "en"], gpu=False)

    # 이미지에서 텍스트 추출
    def extract_text_mpnet_chandra(self, image_path: str) -> str:
        image = Image.open(image_path).convert("RGB")
        result = self.ocr_reader.readtext(np.array(image))
        text = " ".join([txt for _, txt, conf in result if conf > 0.3])
        return text.strip()

    # MPNet 문장 임베딩
    def mpnet_encode(self, sentences: list) -> torch.Tensor:
        encoded_input = self.mpnet_tokenizer(
            sentences,
            padding=True,
            truncation=True,
            max_length=384,
            return_tensors="pt",
        )

        with torch.no_grad():
            model_output = self.mpnet_model(**encoded_input)
            embeddings = model_output.last_hidden_state[:, 0, :]

        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        return embeddings

    # OCR 텍스트를 문장 단위로 정리
    def preprocess_sentences(self, text: str) -> list:
        text = re.sub(r"[^\w\s가-힣\.!?0-9]", " ", text)
        text = re.sub(r"\s+", " ", text).strip()

        text = re.sub(r"(습니다\.|입니다\.|다\.|요\.)", r"\1<SPLIT>", text)
        text = re.sub(r"([\.!?])", r"\1<SPLIT>", text)

        sentences = text.split("<SPLIT>")
        return [s.strip() for s in sentences if len(s.strip()) > 5]

    # 기준 임베딩 1개와 문서 임베딩 여러 개의 코사인 유사도 계산
    def compute_mpnet_similarity(
        self, ref_embedding: torch.Tensor, doc_embeddings: torch.Tensor
    ) -> tuple:
        similarities = torch.nn.functional.cosine_similarity(
            ref_embedding.unsqueeze(0),
            doc_embeddings,
            dim=1,
        )
        max_idx = similarities.argmax()
        return float(similarities[max_idx]), max_idx

    # 기본 분류 파이프라인 (threshold만 사용)
    def classify_childcare_document(self, image_path: str, reference_sentences: list) -> dict:
        # 1) OCR + 문장 분리
        raw_text = self.extract_text_mpnet_chandra(image_path)
        doc_sentences = self.preprocess_sentences(raw_text)

        if len(doc_sentences) == 0:
            return {"verdict": False, "reason": "텍스트 추출 실패"}

        # 2) 기준 문장 + 문서 문장 임베딩
        all_sentences = reference_sentences + doc_sentences
        all_embeddings = self.mpnet_encode(all_sentences)

        ref_embeddings = all_embeddings[: len(reference_sentences)]
        doc_embeddings = all_embeddings[len(reference_sentences) :]

        # 3) 각 기준 문장별 최고 유사도
        best_matches = []
        for i, ref_emb in enumerate(ref_embeddings):
            score, doc_idx = self.compute_mpnet_similarity(ref_emb, doc_embeddings)
            best_matches.append(
                {
                    "ref_idx": i,
                    "score": float(score),
                    "matched_sentence": doc_sentences[doc_idx],
                }
            )

        max_match = max(best_matches, key=lambda x: x["score"])
        verdict = max_match["score"] >= self.threshold

        return {
            "verdict": verdict,
            "max_confidence": round(max_match["score"], 4),
            "best_ref_idx": max_match["ref_idx"],
            "matched_sentence": max_match["matched_sentence"],
            "total_sentences": len(doc_sentences),
            "all_scores": {f"ref_{i}": m["score"] for i, m in enumerate(best_matches)},
            "reference_sentences": reference_sentences,
            "raw_text": raw_text,
        }


# ----------------- 규칙 기반 점수 보정 -----------------

KEYWORDS = ["보육교사", "보육", "영유아보육법", "보육교사자격증", "유치원", "유아교육법", "교육법","유아","영유아"]


def classify_childcare_with_rules(
    classifier: ChildcareDocumentClassifier,
    image_path: str,
    references: list,
    base_threshold: float = 0.70,
    boost_per_hit: float = 0.02,   # 키워드 1개당 가산점
    penalty: float = 0.05,         # 키워드가 하나도 없을 때 감점
) -> dict:
    """
    - MPNet 기반 기본 분류 결과에
    - 보육 핵심 키워드 출현 개수에 따라 점수를 보정.
    """
    # 기본 분류 결과
    result = classifier.classify_childcare_document(image_path, references)

    # 기본 유사도 점수(MPNet 코사인 유사도 최댓값)
    score = result["max_confidence"]

    # OCR 원문(OCR로 읽어낸 문서 전체 텍스트)
    raw_text = result.get("raw_text", "")

    # 키워드 등장 개수 세기
    hit_count = sum(raw_text.count(kw) for kw in KEYWORDS)

    # 점수 가산/감산
    if hit_count > 0:
        score += boost_per_hit * hit_count
    else:
        score -= penalty

    #최종 판정
    verdict = score >= base_threshold

    #결과 딕셔너리에 추가 정보 저장
    result["rule_score"] = round(score, 4)
    result["verdict"] = verdict
    result["keyword_hit_count"] = hit_count
    result["base_threshold"] = base_threshold
    result["boost_per_hit"] = boost_per_hit
    result["penalty"] = penalty

    return result


# ----------------- 사용 예시 -----------------

if __name__ == "__main__":
    # 보육 관련 기준 문장 예시
    REFERENCES = [
        # 공통 타이틀
        "보육교사",

        # 법적 근거 문장
        "영유아보육법에 따라 보육교사의",

        # 급수·직종 표현
        "보육교사 1급",
        "보육교사 2급",
        "보육교사 3급",

        # 교원자격증 쪽(보육과 헷갈릴 수 있는 교육부 양식용)
        "교원 자격증",
        "교육부장관",
    ]

    classifier = ChildcareDocumentClassifier(similarity_threshold=0.70)

    img_path = r"C:\Users\ehjun\Documents\캡스톤디자인\자동문서인식모델\data\보육자격증 아닌것\토익성적.png"

    result = classify_childcare_with_rules(
        classifier,
        image_path=img_path,
        references=REFERENCES,
        base_threshold=0.70,
        boost_per_hit=0.02,   # ← 이름 변경
        penalty=0.05,
    )

    print("🎯 최종 판정:", "보육 관련 문서 ✅" if result["verdict"] else "일반 문서 ❌")
    print("  - base similarity:", result["max_confidence"])
    print("  - rule_score     :", result["rule_score"])
    print("  - keyword hits   :", result["keyword_hit_count"])
    print("  - matched sent   :", result["matched_sentence"])


📥 지정 사전학습 모델 로딩 중...
  - MPNet 문장 임베딩 로딩...


Using CPU. Note: This module is much faster with a GPU.


✅ 모든 모델 로딩 완료!
🎯 최종 판정: 일반 문서 ❌
  - base similarity: 0.5314
  - rule_score     : 0.4814
  - keyword hits   : 0
  - matched sent   : 문서확인번호 1761 2836 1342 6760 진 본 2025 10 24 14 26.
